In [24]:
from utils.config import load_datasets, get_train_dataset_indexes

folder = "../outputs/datasets" # load saved data
# dataset_indexes = get_train_dataset_indexes(folder)
dataset_indexes = ['burn']
datasets = load_datasets(folder=folder, names=dataset_indexes)


In [22]:
from utils.preprocess import preprocess
# ds_train = preprocess(datasets['epileptic'])
# datasets['epileptic']
# ds_train

,ohe__fac_treat_LTG,ohe__fac_gender_M,ohe__fac_learn_dis_Yes,imp__num_dose,imp__num_age,remainder__pid,remainder__event,remainder__time,remainder__time2
794,1.0,0.0,0.0,-0.752997,1.403417,359,0,0,123
104,1.0,0.0,0.0,1.634830,1.507047,42,0,501,691
457,0.0,1.0,0.0,-1.946910,1.060109,179,1,0,91
29,1.0,1.0,0.0,0.440917,0.705640,11,0,245,431
980,1.0,0.0,0.0,0.042946,0.187489,450,1,133,556
...,...,...,...,...,...,...,...,...,...
951,1.0,0.0,0.0,-0.355026,0.695543,430,0,0,1413
478,0.0,1.0,0.0,2.828744,-0.954569,191,0,0,81
1189,0.0,0.0,0.0,-0.617687,1.960363,563,0,0,756
869,1.0,0.0,0.0,-0.355026,-0.190363,391,0,85,182


In [25]:
from synthcity.plugins import Plugins


# https://github.com/vanderschaarlab/synthcity/issues/249
def run_surv_gan(data_loader, device):
    model = Plugins().get("survival_gan", device=device)
    model.fit(data_loader)
    return model
    
def run_surv_vae(data_loader, device):
    model = Plugins().get("survae", device=device)
    model.fit(data_loader)
    return model

In [26]:
from synthcity.metrics.eval_sanity import CloseValuesProbability, DataMismatchScore, CommonRowsProportion, NearestSyntheticNeighborDistance, DistantValuesProbability

def evaluate_model(data_loader, generated_data):
    # Initialize metrics
    close = CloseValuesProbability()
    data_mismatch = DataMismatchScore()
    proportion = CommonRowsProportion()
    nn_distance = NearestSyntheticNeighborDistance()
    distant = DistantValuesProbability()
    
    # Evaluate metrics
    close_val = close.evaluate(data_loader, generated_data)['score']
    mis = data_mismatch.evaluate(data_loader, generated_data)['score']
    prop = proportion.evaluate(data_loader, generated_data)['score']
    nn_dist = nn_distance.evaluate(data_loader, generated_data)['mean']
    dist = distant.evaluate(data_loader, generated_data)['score']
    
    # Determine if metrics are within expected values
    correct = True

    # Define correctness based on descriptions
    if not (0 <= close_val <= 1):
        correct = False
    if not (0 <= mis <= 1):
        correct = False
    if not (0 <= prop <= 1):
        correct = False
    if nn_dist < 0:
        correct = False
    if not (0 <= dist <= 1):
        correct = False
    
    # Prepare results
    results = {
        'close_values': {
            "value": close_val,
        },
        'data_mismatch': {
            "value": mis,
        },
        'proportion': {
            "value": prop,
        },
        'nn_distance': {
            "value": nn_dist,
        },
        'distant_values': {
            "value": dist,
        }
    }
    
    return results, correct


In [27]:
from synthcity.utils.serialization import save_to_file
from utils.config import save_dataset, save_checkpoint, load_checkpoint
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader, TimeSeriesSurvivalDataLoader

import torch
start_index = load_checkpoint()
device = torch.device('cuda')

for index, dataset_index in enumerate(dataset_indexes):
    if index < start_index:
        continue
    print(f"training model on {dataset_index}")
    ds_train = datasets[dataset_index]
    ds_train = preprocess(ds_train)
    data_loader = SurvivalAnalysisDataLoader(ds_train, target_column="remainder__event", time_to_event_column="remainder__time", )
    surv_gan_model = run_surv_gan(data_loader, device)
    surv_vae_model = run_surv_vae(data_loader, device)
    
    save_to_file(f"../outputs/model_outputs/sim_model_{dataset_index}_gan.pkl", surv_gan_model)
    save_to_file(f"../outputs/model_outputs/sim_model_{dataset_index}_vae.pkl", surv_vae_model)
    
    generated_data_gan = surv_gan_model.generate(5000)
    generated_data_vae = surv_vae_model.generate(5000)
    
    _, eval_gan = evaluate_model(data_loader, generated_data_gan)
    _, eval_vae = evaluate_model(data_loader, generated_data_vae)
    
    print(f"training completed, eval: gan:{eval_gan} vae:{eval_vae}")
    
    generated_data_vae = generated_data_vae.dataframe()
    generated_data_gan = generated_data_gan.dataframe()
    
    save_dataset(generated_data_gan, f"{dataset_index}_gan","../outputs/generated_datasets")
    save_dataset(generated_data_vae, f"{dataset_index}_vae","../outputs/generated_datasets")
    save_checkpoint(index)
    

training model on burn


[2024-08-20T17:31:13.416463+0200][129456][CRITICAL] module disabled: C:\Users\johan\anaconda3\envs\research_project_simulation_env\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
INFO:datasets:PyTorch version 1.13.1 available.
[2024-08-20T17:31:15.135904+0200][129456][CRITICAL] module disabled: C:\Users\johan\anaconda3\envs\research_project_simulation_env\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
  9%|▉         | 949/10000 [00:27<04:23, 34.37it/s]
[2024-08-20T17:31:44.455896+0200][129456][CRITICAL] module disabled: C:\Users\johan\anaconda3\envs\research_project_simulation_env\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
[2024-08-20T17:31:44.480896+0200][129456][CRITICAL] module disabled: C:\Users\johan\anaconda3\envs\research_project_simulation_env\lib\site-packages\synthcity\plugins\generic\plugin_goggle.py
 35%|███▌      | 350/1000 [00:07<00:13, 47.19it/s]


training completed, eval: gan:True vae:True
